In [1]:
!pip install kaggle

In [2]:
!mkdir .kaggle

mkdir: cannot create directory ‘.kaggle’: File exists


In [0]:
import json
token = {"username":"lordtt13","key":"2894b36bb67302ce844ba990d4cc4eb7"}
with open('.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json


In [8]:
!kaggle config set -n path -v{/content}


- path is now set to: {/content}


In [0]:
!chmod 600 /root/.kaggle/kaggle.json


In [10]:
!kaggle datasets download -d gti-upm/leapgestrecog

 99% 2.10G/2.13G [00:30<00:00, 78.9MB/s]
100% 2.13G/2.13G [00:30<00:00, 74.3MB/s]


In [12]:
!unzip {/content}/datasets/gti-upm/leapgestrecog/leapgestrecog.zip

Archive:  {/content}/datasets/gti-upm/leapgestrecog/leapgestrecog.zip
  inflating: leapGestRecog/00/01_palm/frame_00_01_0001.png  
  inflating: leapGestRecog/00/01_palm/frame_00_01_0002.png  
  inflating: leapGestRecog/00/01_palm/frame_00_01_0003.png  
  inflating: leapGestRecog/00/01_palm/frame_00_01_0004.png  
  inflating: leapGestRecog/00/01_palm/frame_00_01_0005.png  
  inflating: leapGestRecog/00/01_palm/frame_00_01_0006.png  
  inflating: leapGestRecog/00/01_palm/frame_00_01_0007.png  
  inflating: leapGestRecog/00/01_palm/frame_00_01_0008.png  
  inflating: leapGestRecog/00/01_palm/frame_00_01_0009.png  
  inflating: leapGestRecog/00/01_palm/frame_00_01_0010.png  
  inflating: leapGestRecog/00/01_palm/frame_00_01_0011.png  
  inflating: leapGestRecog/00/01_palm/frame_00_01_0012.png  
  inflating: leapGestRecog/00/01_palm/frame_00_01_0013.png  
  inflating: leapGestRecog/00/01_palm/frame_00_01_0014.png  
  inflating: leapGestRecog/00/01_palm/frame_00_01_0015.png  
  inflating: le

In [0]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
 
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

#preprocess.
from keras.preprocessing.image import ImageDataGenerator

#dl libraraies
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint,EarlyStopping,TensorBoard,CSVLogger,ReduceLROnPlateau,LearningRateScheduler

# specifically for cnn
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,GlobalAveragePooling2D
import tensorflow as tf
import random as rn

# specifically for manipulating zipped images and getting numpy arrays of pixel values of images.
import cv2                  
import numpy as np  
from tqdm import tqdm
import os                   
from random import shuffle  
from zipfile import ZipFile
from PIL import Image

In [0]:
lookup = dict()
reverselookup = dict()
count = 0
for j in os.listdir('../input/leapgestrecog/leapGestRecog/00/'):
    if not j.startswith('.'): # If running this code locally, this is to 
                              # ensure you aren't reading in hidden folders
        lookup[j] = count
        reverselookup[count] = j
        count = count + 1
lookup

In [0]:
x_data = []
y_data = []
IMG_SIZE = 150
datacount = 0 # We'll use this to tally how many images are in our dataset
for i in range(0, 10): # Loop over the ten top-level folders
    for j in os.listdir('../input/leapgestrecog/leapGestRecog/0' + str(i) + '/'):
        if not j.startswith('.'): # Again avoid hidden folders
            count = 0 # To tally images of a given gesture
            for k in os.listdir('../input/leapgestrecog/leapGestRecog/0' + 
                                str(i) + '/' + j + '/'):
                                # Loop over the images
                path = '../input/leapgestrecog/leapGestRecog/0' + str(i) + '/' + j + '/' + k
                img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
                arr = np.array(img)
                x_data.append(arr) 
                count = count + 1
            y_values = np.full((count, 1), lookup[j]) 
            y_data.append(y_values)
            datacount = datacount + count
x_data = np.array(x_data, dtype = 'float32')
y_data = np.array(y_data)
y_data = y_data.reshape(datacount, 1) # Reshape to be the correct size

In [0]:
# check some image
fig,ax=plt.subplots(5,2)
fig.set_size_inches(15,15)
for i in range(5):
    for j in range (2):
        l=rn.randint(0,len(y_data))
        ax[i,j].imshow(x_data[l])
        ax[i,j].set_title(reverselookup[y_data[l,0]])
        
plt.tight_layout()